In [ ]:
# Загрузка геопространственных данных Open street map для использования в QGIS

## Введение
Описываем проблематику - картографические данные нужны, а их нет. Поэтому удобно использовать данные OSM. При этом можно купить, но для ограниченных задач этого не требуется!



## Общие сведения о OSM

## Библиотеки выгрузки данных
Nominatim
Overpass
OSMNX

Здесь же про форматы данных - gpckg

## Алгоритм выгрузки данных

Первым делом для выгрузки данных следует подключить соответствующую библиотеку.

In [1]:
import osmnx as ox

Для удобства дальнейшей работы весь ход загрузки и сохранения данных в формате geopackage был обернут в специальную функцию.

In [2]:
def dwnld_from_osm_by_place(place, tags, fields, file_name, layer_name):
    '''
    Функция загружает произвольные данные из области ограниченной географической территорией 
    и сохраняет их в файле по указанному пути

    place - строковое наименование географической территории

    tags - набор тегов идентифицирующих данные которые следует загрузить

    fields - словарь полей которые следует сохранить для полученного набора. 
    Ключом является имя поля которое следует сохранить в стартовом наборе, 
    значением - итоговое наименование поля в результирующем наборе. 
    Например, {'geometry':'geometry', 'name':'name', 
    'description':'description', 'amenity':'назначение', 'building':'здание'}.
    Поля geometry, name и description являются обязательными, переименовывать их нельзя.

    file_name - полное наименование итогового файла (включая путь).

    layer_name - наименование слоя данных для отображения в легенде QGIS.
    '''
    # Загрузка данных из osmnx
    gdf = ox.geometries_from_place(place, tags)
    # Проверка наличия полей в полученном наборе данных
    for field in list(fields.keys()):
        if not field in gdf.columns:
            del fields[field]
    # Преобразование набора данных
    gdf_s = gdf[list(fields.keys())]
    gdf_s=gdf_s.rename(fields, axis='columns')
    # Сохранение
    gdf_s.to_file(file_name, driver='GPKG', layer=layer_name)

    # Заключение о работе функции - печатаем количество сохраненных записей.
    print(f"Сохранено {gdf.shape[0]} записей для слоя {layer_name}")

Для оценки работы написанной программы загрузим некоторые данные для октябрьского района города Красноярска.

In [5]:
# Наименование географической территории для которой будем выгружать данные:
place = "Октябрьский район, Красноярск, городской округ Красноярск, Красноярский край, Сибирский федеральный округ, Россия"
# Путь к каталогу в котором будем сохранять полученные наборы данных
path = "G:/QGIS/Krsk-2/data/"

Получим сведения о зданиях.

In [6]:
# набор тегов идентифицирующих данные которые следует загрузить
tags = {'building': True}
# словарь полей которые следует сохранить
fields = {'geometry':'geometry',
'name':'name',
'description':'description',
'amenity':'назначение',
'building':'здание',
'alt_name':'альт. имя',
'official_name':'оф. имя',
'abandoned':'заброшено',
'ruins':'разрушено',
'disused':'не используется',
'addr:street':'улица',
'addr:housenumber':'дом',
'rooms':'комнат'}
dwnld_from_osm_by_place(place, tags, fields, path+'bld.gpkg', 'здания')

c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\geometries.py:816: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  for merged_inner_linestring in list(merged_inner_linestrings):
c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\geometries.py:816: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for merged_inner_linestring in list(merged_inner_linestrings):
c:\Users\obsid\.c

Сохранено 9875 записей


Аналогично загрузим данные о улицах и границе района.

In [8]:
# Дороги
tags = {"highway": True}
fields = {'geometry':'geometry',
'name':'name',
'description':'description',
'highway':'HIGHWAY',
'maxspeed':'макс. скорость',
'oneway':'одностороннее', 
'width':'ширина',
'bridge':'мост',
'tunnel':'туннель',
'railway':'ж/д'}
dwnld_from_osm_by_place(place, tags, fields, path+'highways.gpkg', 'дороги')
# Граница района
tags = {'type': 'boundary', 'admin_level':'9', 'name':'Октябрьский район'}
fields = {'geometry':'geometry',
'name':'name',
'description':'description'}
dwnld_from_osm_by_place(place, tags, fields, path+'borders.gpkg', 'граница')

c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


Сохранено 5445 записей


c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
c:\Users\obsid\.conda\envs\ox\lib\site-packages\osmnx\utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


Сохранено 44 записей


## Настройка внешнего вида данных в картах QGIS

## Заключение